Outline:

* Async/Await
* What is aiohttp
* Basic App
* Routing
* JSON/Form/etc
* Running It
* Deploying It
* Start Up/Clean Up
* Websockets
* Extensions/Other Packages
* Questions

# aiohttp: An Introduction

Alec Nikolas Reiter
PyATL Oct. 2017

# async/await

* Successor to `yield from`
* Python 3.5
* `coroutines`
* event loop

# aiohttp

* HTTP Toolkit - client and server
* async/await based
* Minimal

# aiohttp.web

* server
* high level application
* "low level" server

In [2]:
import asyncio
from aiohttp.web import Application, json_response, run_app


async def hello(request):
    return json_response({'message': 'hello'})

app = Application()
app.router.add_get('/', hello)
run_app(app, port=5000, loop=asyncio.get_event_loop())

======== Running on http://0.0.0.0:5000 ========
(Press CTRL+C to quit)


## Routing

* Django-ish and Flask-ish
* Functions and Classes
* Subapplications

### URL Variables

In [ ]:
async def name(request):
    name = request.match_info['name']
    # ...
    
app.router.add_get('/{name:\w+}', name)

### Function Route Handler

In [9]:
async def hello_name(request):
    name = request.match_info['name']
    return json_response({'message': f'hello {name}'})

app = Application()
app.router.add_get('/{name:\w+}', hello_name)
run_app(app, port=5000, loop=asyncio.get_event_loop())

======== Running on http://0.0.0.0:5000 ========
(Press CTRL+C to quit)


### Class Route Handler

In [16]:
from aiohttp.web import View

class Hello(View):
    async def get(self):
        name = self.request.match_info['name']
        return json_response({'message': f'hello {name}'})
    
app = Application()
app.router.add_route('*', '/{name:\w+}', Hello)
run_app(app, port=5000, loop=asyncio.get_event_loop())

<ResourceRoute [*] <DynamicResource  /{name} -> <class '__main__.Hello'>

### Composing Apps Together

In [17]:
parent_app = Application()
parent_app.add_subapp('/name/', app)
run_app(parent_app, port=5000, loop=asyncio.get_event_loop())

======== Running on http://0.0.0.0:5000 ========
(Press CTRL+C to quit)


## Request Data

* `await request.json()`
* `await request.post()`
* `post` also contains file uploads
* `await request.multipart()`

## Running It

* `aiohttp.web.run_app`
* `python -m aiohttp.web -H localhost -P 8080 package.module:factory`

### run_app

* Development Server
* Accepts Event Loop
* Quick and Easy

### CLI

* Development Server
* No `if __name__ == "__main__"`
* Needs factory function
* Unknown args go to factory

In [18]:
def app_factory(argv):   
    app = Application()
    
    if '--hello-name' in argv:
        app.router.add_get('/{name:\w+}', hello_name)
        
    return app

## Deploying It

* Gunicorn
* aiohttp prepackages workers
    * `asyncio.event_loop` based
    * `uvloop` based
* uWSGI has experimental asyncio support

## Signals

* Start up dependencies
* Redis, Message Queue, WSDL, etc
* Accept Coroutines
* Runs *in order*

## Signals Available

* `on_startup`
* `on_teardown`
* `on_cleanup`

## Signals: Buyer Beware

* You're responsible
* aiojobs helper lib

## Websockets

* Socket between client and server
* Allow push from both
* Mostly supported

In [ ]:
from aiohttp.web import WebSocketResponse

def ws_handler(request):
    socket = WebSocketResponse(request)
    await socket.prepare()
    
    async for message in socket:
        # handle message
        
    return socket

## Websockets

* Only one reader allowed
* Multiple pushers
* Cleaned up when client disconnects
* Need cleanup at shutdown

## Other packages

* aiopg/aiomysql
* aioredis/aiomemcache
* aiohttp specific:
    * jinja2
    * debugtoolbar
    * cors
    * security
    * more

# Questions?